# Publishing Feedback

### Setup

In [1]:
# You can set them inline
import os

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Adding Feedback to a Run

You can add feedback to an existing Run programmatically as long as you know the run_id. Let's grab a run_id from the LangSmith UI and add it here.

In [3]:
run_id = "e56b7da1-ea8e-41b1-9219-20613832c3c2"

Now, let's add some continuous feedback

In [4]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-continuous",
  score=7.0,
  comment="This is a sample piece of continuous feedback",
)

/Users/nithurshen/SNU/intro-to-langsmith/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Feedback(id=UUID('a9dd0ec4-fefc-4239-8afe-9f83da2ce09a'), created_at=datetime.datetime(2025, 10, 7, 5, 39, 23, 797189, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 7, 5, 39, 23, 797193, tzinfo=datetime.timezone.utc), run_id=UUID('e56b7da1-ea8e-41b1-9219-20613832c3c2'), trace_id=None, key='sample-continuous', score=7.0, value=None, comment='This is a sample piece of continuous feedback', correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}, user_id=None, user_name=None), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

And now, let's add some categorical feedback too!

In [5]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-categorical",
  value="no",
  comment="This is a sample piece of categorical feedback",
)

Feedback(id=UUID('f9c8f3f0-d469-4f2c-9266-8bd18a6143e4'), created_at=datetime.datetime(2025, 10, 7, 5, 39, 39, 792637, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 7, 5, 39, 39, 792640, tzinfo=datetime.timezone.utc), run_id=UUID('e56b7da1-ea8e-41b1-9219-20613832c3c2'), trace_id=None, key='sample-categorical', score=None, value='no', comment='This is a sample piece of categorical feedback', correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}, user_id=None, user_name=None), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

### Pre-Generating Run IDs for Feedback

![Generate_run_id](../../images/generate_run_id.png)

Using LangChain, we offer the ability to pre-generate and define run IDs, before your code is invoked and the run ID is generated. With this functionality, you're able to access your run ID before initial generation, which can be useful for actions like sending feedback. The example below demonstrates this.

In [6]:
import uuid

pre_defined_run_id = uuid.uuid4()
pre_defined_run_id

UUID('5d74593a-9874-49af-adfd-e566e117f30b')

In [7]:
from langsmith import traceable

@traceable
def foo():
    return "This is a sample Run!"

We are passing in a config with our function call through `langsmith_extra` that contains our pre-defined run_id

In [8]:
foo(langsmith_extra={"run_id": pre_defined_run_id})

'This is a sample Run!'

Now we can directly create feedback on this run!

In [9]:
from langsmith import Client

client = Client()

client.create_feedback(pre_defined_run_id, "user_feedback", score=1)

Feedback(id=UUID('8504f7ee-6b7e-4800-8e27-0fce0b31d26b'), created_at=datetime.datetime(2025, 10, 7, 5, 40, 40, 659569, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 7, 5, 40, 40, 659574, tzinfo=datetime.timezone.utc), run_id=UUID('5d74593a-9874-49af-adfd-e566e117f30b'), trace_id=None, key='user_feedback', score=1, value=None, comment=None, correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}, user_id=None, user_name=None), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

### Pre-signed Feedback URLs

![presigned url](../../images/presigned_url.png)


This can also be helpful for pre-signed feedback URLs. You would want to use these when you can't expose API keys or other secrets to the client, e.g. in a web application. Using a pre-determined run_id LangSmith has an endpoint create_presigned_feedback_token which will create a URL for sending feedback, without the use of secrets required.

In [10]:
pre_signed_url_id = uuid.uuid4()
pre_signed_url_id

UUID('cb94f631-af30-41d1-8e3e-53f607ba5738')

In [11]:
pre_signed_url = client.create_presigned_feedback_token(pre_signed_url_id, "user_presigned_feedback")

print(pre_signed_url)

id=UUID('0af4df75-9657-4afc-b1b6-78a18f30a4d8') url='https://api.smith.langchain.com/feedback/tokens/0af4df75-9657-4afc-b1b6-78a18f30a4d8' expires_at=datetime.datetime(2025, 10, 7, 8, 41, 44, 987189, tzinfo=datetime.timezone.utc)


Here, we can see that even though we haven't created a run yet, we're still able to generate the feedback URL.

Now, let's invoke our chain so the run with that ID is created:

In [12]:
foo(langsmith_extra={"run_id": pre_signed_url_id})

'This is a sample Run!'

Then, once our run is created, we can use the feedback URL to send feedback:

In [13]:
import requests

url_with_score = f"{pre_signed_url.url}?score=1"

response = requests.get(url_with_score)

if response.status_code >= 200 and response.status_code < 300:
    print("Feedback submitted successfully!")
else:
    print("Feedback submission failed!")

Feedback submitted successfully!
